<img src="https://github.com/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/pictures/logo_sizinglab.png?raw=1" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Propeller selection
*Written by Marc Budinger (INSA Toulouse) and Scott Delbecq (ISAE-SUPAERO), Toulouse, France.*

## Design graph 

The following diagram represents the design graph of the propeller’s selection. The max thrust is assumed to be known here.


![DesignGraph](https://github.com/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/pictures/PropellerDesignGraph.png?raw=1)

> **Questions:**
* Give the main sizing problems you are able to detect.
* Propose one or multiple solutions (which can request equation manipulation, addition of design variables, addition of constraints) 
* Orientate the arrows
* Give equations order, inputs/outputs at each step of this part of sizing procedure

> **Responses:**
* Given pitch/D and the F_max, we need an estimation of nD for the calculus of D and, consequently, n. So, we need to create and design variable K_nD as nD = k_nD*(Nd)_max with 0 < k_nD < 1

![DesignGraph](https://github.com/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/pictures/PropellerDesignGraphAvecOrdre.png?raw=1)


* Sizing Procedure :

  - ND = k_nD*ND_max with 0 < k_nD < 1
  - C_t (estimation model)
  - C_p (estimation model)
  - Take off scenario D = sqrt(F_pro_to/(rho_air * (ND^2) * C_t))
  - For hover         D = sqrt(F_pro_hov/(rho_air * (ND^2) * C_t))   
  - n = ND/D
  - T_max = C_p * rho_air * (ND^2) * D^2/W
  - M_pro = M_pro_ref * (D/D_pro_ref)^2 

### Sizing code and optimization

> Exercice: propose a sizing code for the selection of a propeller.


In [ ]:
from math import pi

# Specifications
rho_air=1.18# [kg/m^3] Air density
ND_max=105000./60.*.0254 #[Hz.m] Max speed limit (N.D max) for APC MR propellers

# Reference parameters for scaling laws
D_pro_ref=11.*.0254# [m] Reference propeller diameter
M_pro_ref=0.53*0.0283# [kg] Reference propeller mass

# Assumption

F_pro_to=15.0 #[N] Thrust for 1 propeller during Take Off
F_pro_hov=5.0 #[N] Thrust for 1 propeller during hover

In [ ]:
import pandas as pd
from math import sqrt


# -----------------------
# sizing code
# -----------------------

# Design variables
# ---
    pitch_sur_D  # variable entre 0.3 et 0.6
    k_nd # variable entre 0 et 1

#Equations:
#-----
    ND = k_nD*ND_max

# Regression

    Ct = 4.27e-02 + 1.44e-01 * pitch_sur_D # Linear Regression from previous notebook 04_PropellerLinearRegression-Student.ipynb
    Cp = -1.48e-03 + 9.72e-02 * pitch_sur_D # Linear Regression from previous notebook 04_PropellerLinearRegression-Student.ipynb

# Propeller selection with take-off scenario

    D_pro_to = sqrt(F_pro_to/(rho_air * (ND^2) * Ct)) # From equation Ct = F/rho*N^2*D^3
    n_pro_to = ND/D_to
    W_pro_to = n_pro_to*2*pi
    P_pro_to = Cp * rho_air * (ND**3) * D_to**2 # From equation Cp = P/rho*N^3*D^5 and T = P/W
    T_pro_to = P_pro_to/W_pro_to 
    M_pro_to = M_pro_ref * (D_pro_to/D_pro_ref)**2 # From equation M* = D*^2 et var* = var/var_ref


# Propeller torque & speed for hover
    D_pro_hov = sqrt(F_pro_hov/(rho_air * (ND^2) * Ct))  # From equation Ct = F/rho*N^2*D^3 
    n_pro_hov = ND/D_hov
    W_pro_hov = n_pro_hov*2*pi
    P_pro_hov = Cp * rho_air * (ND**3) * D_hov**2 # From equation Cp = P/rho*N^3*D^5 and T = P/W
    T_pro_hov = P_pro_hov/W_pro_hov
    M_pro_hov = M_pro_ref * (D_pro_hov/D_pro_ref)**2 # From equation M* = D*^2 et var* = var/var_ref
